In [1]:
from database.database import DatabaseProcessor
from Embedders import get_embedder
from dotenv import load_dotenv
import os

load_dotenv('.env')

True

In [2]:
db_params = {
    'dbname': os.getenv('DB_NAME') ,
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASSWORD'),
    'host': os.getenv('DB_HOST'),
    'port': os.getenv('DB_PORT')
}
db = DatabaseProcessor(db_params)
print(db.db_params)
embedder = get_embedder(model_name='BAAI/bge-small-en', device='mps', normalize=True)


{'dbname': 'test', 'user': 'bbasseri', 'password': 'citeline25', 'host': 'localhost', 'port': '5432'}


In [3]:
# Get 4096 chunks from the database
import psycopg2

def get_chunks(num: int):
    conn = psycopg2.connect(**db.db_params)
    cur = conn.cursor()
    cur.execute("SELECT text FROM chunks LIMIT %s", (num,))
    rows = cur.fetchall()
    cur.close()
    conn.close()
    return [row[0] for row in rows]

In [4]:
batch_size = 4096

while batch_size > 1:
    try:
        chunks = get_chunks(batch_size)
        print(f"Got {len(chunks)} chunks")
        embeddings = embedder(chunks)
        print(embeddings.shape)
        break
    except Exception as e:
        print(e)
        batch_size = batch_size // 2
        print(f"Reducing batch size to {batch_size}")
        continue


Got 4096 chunks


Process SpawnProcess-1:
Traceback (most recent call last):
  File "/Users/benjaminbasseri/miniforge3/envs/citeline/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/benjaminbasseri/miniforge3/envs/citeline/lib/python3.11/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/benjaminbasseri/miniforge3/envs/citeline/lib/python3.11/site-packages/sentence_transformers/SentenceTransformer.py", line 993, in _encode_multi_process_worker
    embeddings = model.encode(
                 ^^^^^^^^^^^^^
  File "/Users/benjaminbasseri/miniforge3/envs/citeline/lib/python3.11/site-packages/sentence_transformers/SentenceTransformer.py", line 623, in encode
    out_features = self.forward(features, **kwargs)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/benjaminbasseri/miniforge3/envs/citeline/lib/python3.11/site-packages/sentence_transformers/SentenceTransformer.py", line 690, in fo

KeyboardInterrupt: 